In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.set_printoptions(precision=3, suppress=True)
from tensorflow import keras
from tensorflow.keras import layers
import array as arr

In [ ]:
from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("current", graph.GetTitle())
            if searching != None:
                a = graph.GetTitle()[43:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                x_title = event.GetXaxis()
                if (str(x_title))[22:-1] != 'ns':
                    print('Units have changed')
                    break
                
    return pulsed_array, x, y

def collect_efield(files_tot, position):
    each_field = arr.array('f', [])
    for file in files_tot:
        field_values = arr.array('f', [])
        for f, p in zip(file, position):
            myFile = TFile.Open(f, "read")
            data = myFile.Get("ElectricFieldReader")
            detector = data.Get("detector1")
            efield = detector.Get("field1d_z")
            no_bins = efield.GetNbinsX()
            bin_size = 0.1/no_bins
            
            pos = p*10**(-3)
                
            placement = -0.05
            for bin in range(no_bins):  
                placement += bin_size
                if placement >= pos:
                    field_strength = arr.array('f', [efield.GetBinContent(bin)])
                    field_values = np.concatenate((field_values, field_strength))
                    break
                    
        each_field = np.concatenate((each_field, field_values))
    each_field = each_field.reshape((len(files_tot)*len(position), 1))
        
    return each_field

def array_data(field_x, field_y, time_vals):
    field_data = arr.array('f', [])
    for x_data, y_data in zip(field_x, field_y):
        data_tot = arr.array('f', [])
        for x, y in zip(x_data, y_data):
            y_array = arr.array('f', [])
            for t in time_vals:
                if x[-1] < t:
                    y_array.append(0)
                    if len(y_array) == len(time_vals):
                        data_tot = np.concatenate((data_tot, y_array))
                
                else:
                    for i in range(len(x)):
                        if x[i] >= t and x[i-1] < t:
                            y_array.append(y[i])
                            if len(y_array) == len(time_vals):
                                data_tot = np.concatenate((data_tot, y_array))
                            break
                        else:
                            continue
    
        field_data = np.concatenate((field_data, data_tot))
    field_data = field_data.reshape((len(field_x)*9, len(time_vals)))

    return field_data

positions = [-40, -30, -20, -10, 0, 10, 20, 30, 40] 

files_tot = [['root_files/lin_-1000_0_40.root', 'root_files/lin_-1000_0_30.root', 'root_files/lin_-1000_0_20.root', 'root_files/lin_-1000_0_10.root', 'root_files/lin_-1000_0_0.root', 'root_files/lin_-1000_0_-10.root', 'root_files/lin_-1000_0_-20.root', 'root_files/lin_-1000_0_-30.root', 'root_files/lin_-1000_0_-40.root'], ['root_files/lin_-1000_2_40.root', 'root_files/lin_-1000_2_30.root', 'root_files/lin_-1000_2_20.root', 'root_files/lin_-1000_2_10.root', 'root_files/lin_-1000_2_0.root', 'root_files/lin_-1000_2_-10.root', 'root_files/lin_-1000_2_-20.root', 'root_files/lin_-1000_2_-30.root', 'root_files/lin_-1000_2_-40.root'], ['root_files/lin_-1000_100_40.root', 'root_files/lin_-1000_100_30.root', 'root_files/lin_-1000_100_20.root', 'root_files/lin_-1000_100_10.root', 'root_files/lin_-1000_100_0.root', 'root_files/lin_-1000_100_-10.root', 'root_files/lin_-1000_100_-20.root', 'root_files/lin_-1000_100_-30.root', 'root_files/lin_-1000_100_-40.root'], ['root_files/lin_-1000_500_40.root', 'root_files/lin_-1000_500_30.root', 'root_files/lin_-1000_500_20.root', 'root_files/lin_-1000_500_10.root', 'root_files/lin_-1000_500_0.root', 'root_files/lin_-1000_500_-10.root', 'root_files/lin_-1000_500_-20.root', 'root_files/lin_-1000_500_-30.root', 'root_files/lin_-1000_500_-40.root'], ['root_files/lin_-500_0_40.root', 'root_files/lin_-500_0_30.root', 'root_files/lin_-500_0_20.root', 'root_files/lin_-500_0_10.root', 'root_files/lin_-500_0_0.root', 'root_files/lin_-500_0_-10.root', 'root_files/lin_-500_0_-20.root', 'root_files/lin_-500_0_-30.root', 'root_files/lin_-500_0_-40.root'], ['root_files/lin_-500_2_40.root', 'root_files/lin_-500_2_30.root', 'root_files/lin_-500_2_20.root', 'root_files/lin_-500_2_10.root', 'root_files/lin_-500_2_0.root', 'root_files/lin_-500_2_-10.root', 'root_files/lin_-500_2_-20.root', 'root_files/lin_-500_2_-30.root', 'root_files/lin_-500_2_-40.root'], ['root_files/lin_-500_10_40.root', 'root_files/lin_-500_10_30.root', 'root_files/lin_-500_10_20.root', 'root_files/lin_-500_10_10.root', 'root_files/lin_-500_10_0.root', 'root_files/lin_-500_10_-10.root', 'root_files/lin_-500_10_-20.root', 'root_files/lin_-500_10_-30.root', 'root_files/lin_-500_10_-40.root'], ['root_files/lin_-500_100_40.root', 'root_files/lin_-500_100_30.root', 'root_files/lin_-500_100_20.root', 'root_files/lin_-500_100_10.root', 'root_files/lin_-500_100_0.root', 'root_files/lin_-500_100_-10.root', 'root_files/lin_-500_100_-20.root', 'root_files/lin_-500_100_-30.root', 'root_files/lin_-500_100_-40.root'], ['root_files/lin_-300_0_40.root', 'root_files/lin_-300_0_30.root', 'root_files/lin_-300_0_20.root', 'root_files/lin_-300_0_10.root', 'root_files/lin_-300_0_0.root', 'root_files/lin_-300_0_-10.root', 'root_files/lin_-300_0_-20.root', 'root_files/lin_-300_0_-30.root', 'root_files/lin_-300_0_-40.root'], ['root_files/lin_-300_2_40.root', 'root_files/lin_-300_2_30.root', 'root_files/lin_-300_2_20.root', 'root_files/lin_-300_2_10.root', 'root_files/lin_-300_2_0.root', 'root_files/lin_-300_2_-10.root', 'root_files/lin_-300_2_-20.root', 'root_files/lin_-300_2_-30.root', 'root_files/lin_-300_2_-40.root'], ['root_files/lin_-300_10_40.root', 'root_files/lin_-300_10_30.root', 'root_files/lin_-300_10_20.root', 'root_files/lin_-300_10_10.root', 'root_files/lin_-300_10_0.root', 'root_files/lin_-300_10_-10.root', 'root_files/lin_-300_10_-20.root', 'root_files/lin_-300_10_-30.root', 'root_files/lin_-300_10_-40.root'], ['root_files/lin_-300_100_40.root', 'root_files/lin_-300_100_30.root', 'root_files/lin_-300_100_20.root', 'root_files/lin_-300_100_10.root', 'root_files/lin_-300_100_0.root', 'root_files/lin_-300_100_-10.root', 'root_files/lin_-300_100_-20.root', 'root_files/lin_-300_100_-30.root', 'root_files/lin_-300_100_-40.root'], ['root_files/lin_-100_0_40.root', 'root_files/lin_-100_0_30.root', 'root_files/lin_-100_0_20.root', 'root_files/lin_-100_0_10.root', 'root_files/lin_-100_0_0.root', 'root_files/lin_-100_0_-10.root', 'root_files/lin_-100_0_-20.root', 'root_files/lin_-100_0_-30.root', 'root_files/lin_-100_0_-40.root'], ['root_files/lin_-100_2_40.root', 'root_files/lin_-100_2_30.root', 'root_files/lin_-100_2_20.root', 'root_files/lin_-100_2_10.root', 'root_files/lin_-100_2_0.root', 'root_files/lin_-100_2_-10.root', 'root_files/lin_-100_2_-20.root', 'root_files/lin_-100_2_-30.root', 'root_files/lin_-100_2_-40.root'], ['root_files/lin_-100_10_40.root', 'root_files/lin_-100_10_30.root', 'root_files/lin_-100_10_20.root', 'root_files/lin_-100_10_10.root', 'root_files/lin_-100_10_0.root', 'root_files/lin_-100_10_-10.root', 'root_files/lin_-100_10_-20.root', 'root_files/lin_-100_10_-30.root', 'root_files/lin_-100_10_-40.root'], ['root_files/lin_-100_100_40.root', 'root_files/lin_-100_100_30.root', 'root_files/lin_-100_100_20.root', 'root_files/lin_-100_100_10.root', 'root_files/lin_-100_100_0.root', 'root_files/lin_-100_100_-10.root', 'root_files/lin_-100_100_-20.root', 'root_files/lin_-100_100_-30.root', 'root_files/lin_-100_100_-40.root'], ['root_files/lin_-70_0_40.root', 'root_files/lin_-70_0_30.root', 'root_files/lin_-70_0_20.root', 'root_files/lin_-70_0_10.root', 'root_files/lin_-70_0_0.root', 'root_files/lin_-70_0_-10.root', 'root_files/lin_-70_0_-20.root', 'root_files/lin_-70_0_-30.root', 'root_files/lin_-70_0_-40.root'], ['root_files/lin_-70_2_40.root', 'root_files/lin_-70_2_30.root', 'root_files/lin_-70_2_20.root', 'root_files/lin_-70_2_10.root', 'root_files/lin_-70_2_0.root', 'root_files/lin_-70_2_-10.root', 'root_files/lin_-70_2_-20.root', 'root_files/lin_-70_2_-30.root', 'root_files/lin_-70_2_-40.root'], ['root_files/lin_-70_10_40.root', 'root_files/lin_-70_10_30.root', 'root_files/lin_-70_10_20.root', 'root_files/lin_-70_10_10.root', 'root_files/lin_-70_10_0.root', 'root_files/lin_-70_10_-10.root', 'root_files/lin_-70_10_-20.root', 'root_files/lin_-70_10_-30.root', 'root_files/lin_-70_10_-40.root'], ['root_files/lin_-30_0_40.root', 'root_files/lin_-30_0_30.root', 'root_files/lin_-30_0_20.root', 'root_files/lin_-30_0_10.root', 'root_files/lin_-30_0_0.root', 'root_files/lin_-30_0_-10.root', 'root_files/lin_-30_0_-20.root', 'root_files/lin_-30_0_-30.root', 'root_files/lin_-30_0_-40.root'], ['root_files/lin_-30_2_40.root', 'root_files/lin_-30_2_30.root', 'root_files/lin_-30_2_20.root', 'root_files/lin_-30_2_10.root', 'root_files/lin_-30_2_0.root', 'root_files/lin_-30_2_-10.root', 'root_files/lin_-30_2_-20.root', 'root_files/lin_-30_2_-30.root', 'root_files/lin_-30_2_-40.root'], ['root_files/lin_-30_10_40.root', 'root_files/lin_-30_10_30.root', 'root_files/lin_-30_10_20.root', 'root_files/lin_-30_10_10.root', 'root_files/lin_-30_10_0.root', 'root_files/lin_-30_10_-10.root', 'root_files/lin_-30_10_-20.root', 'root_files/lin_-30_10_-30.root', 'root_files/lin_-30_10_-40.root'], ['root_files/lin_-10_0_40.root', 'root_files/lin_-10_0_30.root', 'root_files/lin_-10_0_20.root', 'root_files/lin_-10_0_10.root', 'root_files/lin_-10_0_0.root', 'root_files/lin_-10_0_-10.root', 'root_files/lin_-10_0_-20.root', 'root_files/lin_-10_0_-30.root', 'root_files/lin_-10_0_-40.root'], ['root_files/lin_-10_2_40.root', 'root_files/lin_-10_2_30.root', 'root_files/lin_-10_2_20.root', 'root_files/lin_-10_2_10.root', 'root_files/lin_-10_2_0.root', 'root_files/lin_-10_2_-10.root', 'root_files/lin_-10_2_-20.root', 'root_files/lin_-10_2_-30.root', 'root_files/lin_-10_2_-40.root'], ['root_files/lin_-10_10_40.root', 'root_files/lin_-10_10_30.root', 'root_files/lin_-10_10_20.root', 'root_files/lin_-10_10_10.root', 'root_files/lin_-10_10_0.root', 'root_files/lin_-10_10_-10.root', 'root_files/lin_-10_10_-20.root', 'root_files/lin_-10_10_-30.root', 'root_files/lin_-10_10_-40.root'], ['root_files/lin_-5_0_40.root', 'root_files/lin_-5_0_30.root', 'root_files/lin_-5_0_20.root', 'root_files/lin_-5_0_10.root', 'root_files/lin_-5_0_0.root', 'root_files/lin_-5_0_-10.root', 'root_files/lin_-5_0_-20.root', 'root_files/lin_-5_0_-30.root', 'root_files/lin_-5_0_-40.root'], ['root_files/lin_-5_2_40.root', 'root_files/lin_-5_2_30.root', 'root_files/lin_-5_2_20.root', 'root_files/lin_-5_2_10.root', 'root_files/lin_-5_2_0.root', 'root_files/lin_-5_2_-10.root', 'root_files/lin_-5_2_-20.root', 'root_files/lin_-5_2_-30.root', 'root_files/lin_-5_2_-40.root'], ['root_files/lin_-2_0_40.root', 'root_files/lin_-2_0_30.root', 'root_files/lin_-2_0_20.root', 'root_files/lin_-2_0_10.root', 'root_files/lin_-2_0_0.root', 'root_files/lin_-2_0_-10.root', 'root_files/lin_-2_0_-20.root', 'root_files/lin_-2_0_-30.root', 'root_files/lin_-2_0_-40.root'], ['root_files/lin_-2_2_40.root', 'root_files/lin_-2_2_30.root', 'root_files/lin_-2_2_20.root', 'root_files/lin_-2_2_10.root', 'root_files/lin_-2_2_0.root', 'root_files/lin_-2_2_-10.root', 'root_files/lin_-2_2_-20.root', 'root_files/lin_-2_2_-30.root', 'root_files/lin_-2_2_-40.root'], ['root_files/lin_2_0_40.root', 'root_files/lin_2_0_30.root', 'root_files/lin_2_0_20.root', 'root_files/lin_2_0_10.root', 'root_files/lin_2_0_0.root', 'root_files/lin_2_0_-10.root', 'root_files/lin_2_0_-20.root', 'root_files/lin_2_0_-30.root', 'root_files/lin_2_0_-40.root'], ['root_files/lin_2_2_40.root', 'root_files/lin_2_2_30.root', 'root_files/lin_2_2_20.root', 'root_files/lin_2_2_10.root', 'root_files/lin_2_2_0.root', 'root_files/lin_2_2_-10.root', 'root_files/lin_2_2_-20.root', 'root_files/lin_2_2_-30.root', 'root_files/lin_2_2_-40.root'], ['root_files/lin_5_0_40.root', 'root_files/lin_5_0_30.root', 'root_files/lin_5_0_20.root', 'root_files/lin_5_0_10.root', 'root_files/lin_5_0_0.root', 'root_files/lin_5_0_-10.root', 'root_files/lin_5_0_-20.root', 'root_files/lin_5_0_-30.root', 'root_files/lin_5_0_-40.root'], ['root_files/lin_5_2_40.root', 'root_files/lin_5_2_30.root', 'root_files/lin_5_2_20.root', 'root_files/lin_5_2_10.root', 'root_files/lin_5_2_0.root', 'root_files/lin_5_2_-10.root', 'root_files/lin_5_2_-20.root', 'root_files/lin_5_2_-30.root', 'root_files/lin_5_2_-40.root'], ['root_files/lin_10_0_40.root', 'root_files/lin_10_0_30.root', 'root_files/lin_10_0_20.root', 'root_files/lin_10_0_10.root', 'root_files/lin_10_0_0.root', 'root_files/lin_10_0_-10.root', 'root_files/lin_10_0_-20.root', 'root_files/lin_10_0_-30.root', 'root_files/lin_10_0_-40.root'], ['root_files/lin_10_2_40.root', 'root_files/lin_10_2_30.root', 'root_files/lin_10_2_20.root', 'root_files/lin_10_2_10.root', 'root_files/lin_10_2_0.root', 'root_files/lin_10_2_-10.root', 'root_files/lin_10_2_-20.root', 'root_files/lin_10_2_-30.root', 'root_files/lin_10_2_-40.root'], ['root_files/lin_10_10_40.root', 'root_files/lin_10_10_30.root', 'root_files/lin_10_10_20.root', 'root_files/lin_10_10_10.root', 'root_files/lin_10_10_0.root', 'root_files/lin_10_10_-10.root', 'root_files/lin_10_10_-20.root', 'root_files/lin_10_10_-30.root', 'root_files/lin_10_10_-40.root'], ['root_files/lin_30_0_40.root', 'root_files/lin_30_0_30.root', 'root_files/lin_30_0_20.root', 'root_files/lin_30_0_10.root', 'root_files/lin_30_0_0.root', 'root_files/lin_30_0_-10.root', 'root_files/lin_30_0_-20.root', 'root_files/lin_30_0_-30.root', 'root_files/lin_30_0_-40.root'], ['root_files/lin_30_2_40.root', 'root_files/lin_30_2_30.root', 'root_files/lin_30_2_20.root', 'root_files/lin_30_2_10.root', 'root_files/lin_30_2_0.root', 'root_files/lin_30_2_-10.root', 'root_files/lin_30_2_-20.root', 'root_files/lin_30_2_-30.root', 'root_files/lin_30_2_-40.root'], ['root_files/lin_30_10_40.root', 'root_files/lin_30_10_30.root', 'root_files/lin_30_10_20.root', 'root_files/lin_30_10_10.root', 'root_files/lin_30_10_0.root', 'root_files/lin_30_10_-10.root', 'root_files/lin_30_10_-20.root', 'root_files/lin_30_10_-30.root', 'root_files/lin_30_10_-40.root'], ['root_files/lin_70_0_40.root', 'root_files/lin_70_0_30.root', 'root_files/lin_70_0_20.root', 'root_files/lin_70_0_10.root', 'root_files/lin_70_0_0.root', 'root_files/lin_70_0_-10.root', 'root_files/lin_70_0_-20.root', 'root_files/lin_70_0_-30.root', 'root_files/lin_70_0_-40.root'], ['root_files/lin_70_2_40.root', 'root_files/lin_70_2_30.root', 'root_files/lin_70_2_20.root', 'root_files/lin_70_2_10.root', 'root_files/lin_70_2_0.root', 'root_files/lin_70_2_-10.root', 'root_files/lin_70_2_-20.root', 'root_files/lin_70_2_-30.root', 'root_files/lin_70_2_-40.root'], ['root_files/lin_70_10_40.root', 'root_files/lin_70_10_30.root', 'root_files/lin_70_10_20.root', 'root_files/lin_70_10_10.root', 'root_files/lin_70_10_0.root', 'root_files/lin_70_10_-10.root', 'root_files/lin_70_10_-20.root', 'root_files/lin_70_10_-30.root', 'root_files/lin_70_10_-40.root'], ['root_files/lin_100_0_40.root', 'root_files/lin_100_0_30.root', 'root_files/lin_100_0_20.root', 'root_files/lin_100_0_10.root', 'root_files/lin_100_0_0.root', 'root_files/lin_100_0_-10.root', 'root_files/lin_100_0_-20.root', 'root_files/lin_100_0_-30.root', 'root_files/lin_100_0_-40.root'], ['root_files/lin_100_2_40.root', 'root_files/lin_100_2_30.root', 'root_files/lin_100_2_20.root', 'root_files/lin_100_2_10.root', 'root_files/lin_100_2_0.root', 'root_files/lin_100_2_-10.root', 'root_files/lin_100_2_-20.root', 'root_files/lin_100_2_-30.root', 'root_files/lin_100_2_-40.root'], ['root_files/lin_100_10_40.root', 'root_files/lin_100_10_30.root', 'root_files/lin_100_10_20.root', 'root_files/lin_100_10_10.root', 'root_files/lin_100_10_0.root', 'root_files/lin_100_10_-10.root', 'root_files/lin_100_10_-20.root', 'root_files/lin_100_10_-30.root', 'root_files/lin_100_10_-40.root'], ['root_files/lin_100_100_40.root', 'root_files/lin_100_100_30.root', 'root_files/lin_100_100_20.root', 'root_files/lin_100_100_10.root', 'root_files/lin_100_100_0.root', 'root_files/lin_100_100_-10.root', 'root_files/lin_100_100_-20.root', 'root_files/lin_100_100_-30.root', 'root_files/lin_100_100_-40.root'], ['root_files/lin_300_0_40.root', 'root_files/lin_300_0_30.root', 'root_files/lin_300_0_20.root', 'root_files/lin_300_0_10.root', 'root_files/lin_300_0_0.root', 'root_files/lin_300_0_-10.root', 'root_files/lin_300_0_-20.root', 'root_files/lin_300_0_-30.root', 'root_files/lin_300_0_-40.root'], ['root_files/lin_300_2_40.root', 'root_files/lin_300_2_30.root', 'root_files/lin_300_2_20.root', 'root_files/lin_300_2_10.root', 'root_files/lin_300_2_0.root', 'root_files/lin_300_2_-10.root', 'root_files/lin_300_2_-20.root', 'root_files/lin_300_2_-30.root', 'root_files/lin_300_2_-40.root'], ['root_files/lin_300_10_40.root', 'root_files/lin_300_10_30.root', 'root_files/lin_300_10_20.root', 'root_files/lin_300_10_10.root', 'root_files/lin_300_10_0.root', 'root_files/lin_300_10_-10.root', 'root_files/lin_300_10_-20.root', 'root_files/lin_300_10_-30.root', 'root_files/lin_300_10_-40.root'], ['root_files/lin_500_0_40.root', 'root_files/lin_500_0_30.root', 'root_files/lin_500_0_20.root', 'root_files/lin_500_0_10.root', 'root_files/lin_500_0_0.root', 'root_files/lin_500_0_-10.root', 'root_files/lin_500_0_-20.root', 'root_files/lin_500_0_-30.root', 'root_files/lin_500_0_-40.root'], ['root_files/lin_500_2_40.root', 'root_files/lin_500_2_30.root', 'root_files/lin_500_2_20.root', 'root_files/lin_500_2_10.root', 'root_files/lin_500_2_0.root', 'root_files/lin_500_2_-10.root', 'root_files/lin_500_2_-20.root', 'root_files/lin_500_2_-30.root', 'root_files/lin_500_2_-40.root'], ['root_files/lin_500_10_40.root', 'root_files/lin_500_10_30.root', 'root_files/lin_500_10_20.root', 'root_files/lin_500_10_10.root', 'root_files/lin_500_10_0.root', 'root_files/lin_500_10_-10.root', 'root_files/lin_500_10_-20.root', 'root_files/lin_500_10_-30.root', 'root_files/lin_500_10_-40.root'], ['root_files/lin_500_100_40.root', 'root_files/lin_500_100_30.root', 'root_files/lin_500_100_20.root', 'root_files/lin_500_100_10.root', 'root_files/lin_500_100_0.root', 'root_files/lin_500_100_-10.root', 'root_files/lin_500_100_-20.root', 'root_files/lin_500_100_-30.root', 'root_files/lin_500_100_-40.root'], ['root_files/lin_500_500_40.root', 'root_files/lin_500_500_30.root', 'root_files/lin_500_500_20.root', 'root_files/lin_500_500_10.root', 'root_files/lin_500_500_0.root', 'root_files/lin_500_500_-10.root', 'root_files/lin_500_500_-20.root', 'root_files/lin_500_500_-30.root', 'root_files/lin_500_500_-40.root'], ['root_files/lin_1000_0_40.root', 'root_files/lin_1000_0_30.root', 'root_files/lin_1000_0_20.root', 'root_files/lin_1000_0_10.root', 'root_files/lin_1000_0_0.root', 'root_files/lin_1000_0_-10.root', 'root_files/lin_1000_0_-20.root', 'root_files/lin_1000_0_-30.root', 'root_files/lin_1000_0_-40.root'], ['root_files/lin_1000_2_40.root', 'root_files/lin_1000_2_30.root', 'root_files/lin_1000_2_20.root', 'root_files/lin_1000_2_10.root', 'root_files/lin_1000_2_0.root', 'root_files/lin_1000_2_-10.root', 'root_files/lin_1000_2_-20.root', 'root_files/lin_1000_2_-30.root', 'root_files/lin_1000_2_-40.root'], ['root_files/lin_1000_10_40.root', 'root_files/lin_1000_10_30.root', 'root_files/lin_1000_10_20.root', 'root_files/lin_1000_10_10.root', 'root_files/lin_1000_10_0.root', 'root_files/lin_1000_10_-10.root', 'root_files/lin_1000_10_-20.root', 'root_files/lin_1000_10_-30.root', 'root_files/lin_1000_10_-40.root'], ['root_files/lin_1000_100_40.root', 'root_files/lin_1000_100_30.root', 'root_files/lin_1000_100_20.root', 'root_files/lin_1000_100_10.root', 'root_files/lin_1000_100_0.root', 'root_files/lin_1000_100_-10.root', 'root_files/lin_1000_100_-20.root', 'root_files/lin_1000_100_-30.root', 'root_files/lin_1000_100_-40.root'], ['root_files/lin_1000_500_40.root', 'root_files/lin_1000_500_30.root', 'root_files/lin_1000_500_20.root', 'root_files/lin_1000_500_10.root', 'root_files/lin_1000_500_0.root', 'root_files/lin_1000_500_-10.root', 'root_files/lin_1000_500_-20.root', 'root_files/lin_1000_500_-30.root', 'root_files/lin_1000_500_-40.root']] + [['root_files/lin_850_0_40.root', 'root_files/lin_850_0_30.root', 'root_files/lin_850_0_20.root', 'root_files/lin_850_0_10.root', 'root_files/lin_850_0_0.root', 'root_files/lin_850_0_-10.root', 'root_files/lin_850_0_-20.root', 'root_files/lin_850_0_-30.root', 'root_files/lin_850_0_-40.root'], ['root_files/lin_850_100_40.root', 'root_files/lin_850_100_30.root', 'root_files/lin_850_100_20.root', 'root_files/lin_850_100_10.root', 'root_files/lin_850_100_0.root', 'root_files/lin_850_100_-10.root', 'root_files/lin_850_100_-20.root', 'root_files/lin_850_100_-30.root', 'root_files/lin_850_100_-40.root'], ['root_files/lin_850_500_40.root', 'root_files/lin_850_500_30.root', 'root_files/lin_850_500_20.root', 'root_files/lin_850_500_10.root', 'root_files/lin_850_500_0.root', 'root_files/lin_850_500_-10.root', 'root_files/lin_850_500_-20.root', 'root_files/lin_850_500_-30.root', 'root_files/lin_850_500_-40.root'], ['root_files/lin_850_700_40.root', 'root_files/lin_850_700_30.root', 'root_files/lin_850_700_20.root', 'root_files/lin_850_700_10.root', 'root_files/lin_850_700_0.root', 'root_files/lin_850_700_-10.root', 'root_files/lin_850_700_-20.root', 'root_files/lin_850_700_-30.root', 'root_files/lin_850_700_-40.root'], ['root_files/lin_700_0_40.root', 'root_files/lin_700_0_30.root', 'root_files/lin_700_0_20.root', 'root_files/lin_700_0_10.root', 'root_files/lin_700_0_0.root', 'root_files/lin_700_0_-10.root', 'root_files/lin_700_0_-20.root', 'root_files/lin_700_0_-30.root', 'root_files/lin_700_0_-40.root'], ['root_files/lin_700_100_40.root', 'root_files/lin_700_100_30.root', 'root_files/lin_700_100_20.root', 'root_files/lin_700_100_10.root', 'root_files/lin_700_100_0.root', 'root_files/lin_700_100_-10.root', 'root_files/lin_700_100_-20.root', 'root_files/lin_700_100_-30.root', 'root_files/lin_700_100_-40.root'], ['root_files/lin_700_500_40.root', 'root_files/lin_700_500_30.root', 'root_files/lin_700_500_20.root', 'root_files/lin_700_500_10.root', 'root_files/lin_700_500_0.root', 'root_files/lin_700_500_-10.root', 'root_files/lin_700_500_-20.root', 'root_files/lin_700_500_-30.root', 'root_files/lin_700_500_-40.root'], ['root_files/lin_700_700_40.root', 'root_files/lin_700_700_30.root', 'root_files/lin_700_700_20.root', 'root_files/lin_700_700_10.root', 'root_files/lin_700_700_0.root', 'root_files/lin_700_700_-10.root', 'root_files/lin_700_700_-20.root', 'root_files/lin_700_700_-30.root', 'root_files/lin_700_700_-40.root']] + [['root_files/lin_-700_0_40.root', 'root_files/lin_-700_0_30.root', 'root_files/lin_-700_0_20.root', 'root_files/lin_-700_0_10.root', 'root_files/lin_-700_0_0.root', 'root_files/lin_-700_0_-10.root', 'root_files/lin_-700_0_-20.root', 'root_files/lin_-700_0_-30.root', 'root_files/lin_-700_0_-40.root'], ['root_files/lin_-700_100_40.root', 'root_files/lin_-700_100_30.root', 'root_files/lin_-700_100_20.root', 'root_files/lin_-700_100_10.root', 'root_files/lin_-700_100_0.root', 'root_files/lin_-700_100_-10.root', 'root_files/lin_-700_100_-20.root', 'root_files/lin_-700_100_-30.root', 'root_files/lin_-700_100_-40.root'], ['root_files/lin_-700_500_40.root', 'root_files/lin_-700_500_30.root', 'root_files/lin_-700_500_20.root', 'root_files/lin_-700_500_10.root', 'root_files/lin_-700_500_0.root', 'root_files/lin_-700_500_-10.root', 'root_files/lin_-700_500_-20.root', 'root_files/lin_-700_500_-30.root', 'root_files/lin_-700_500_-40.root'], ['root_files/lin_-700_700_40.root', 'root_files/lin_-700_700_30.root', 'root_files/lin_-700_700_20.root', 'root_files/lin_-700_700_10.root', 'root_files/lin_-700_700_0.root', 'root_files/lin_-700_700_-10.root', 'root_files/lin_-700_700_-20.root', 'root_files/lin_-700_700_-30.root', 'root_files/lin_-700_700_-40.root'], ['root_files/lin_-850_0_40.root', 'root_files/lin_-850_0_30.root', 'root_files/lin_-850_0_20.root', 'root_files/lin_-850_0_10.root', 'root_files/lin_-850_0_0.root', 'root_files/lin_-850_0_-10.root', 'root_files/lin_-850_0_-20.root', 'root_files/lin_-850_0_-30.root', 'root_files/lin_-850_0_-40.root'], ['root_files/lin_-850_100_40.root', 'root_files/lin_-850_100_30.root', 'root_files/lin_-850_100_20.root', 'root_files/lin_-850_100_10.root', 'root_files/lin_-850_100_0.root', 'root_files/lin_-850_100_-10.root', 'root_files/lin_-850_100_-20.root', 'root_files/lin_-850_100_-30.root', 'root_files/lin_-850_100_-40.root'], ['root_files/lin_-850_500_40.root', 'root_files/lin_-850_500_30.root', 'root_files/lin_-850_500_20.root', 'root_files/lin_-850_500_10.root', 'root_files/lin_-850_500_0.root', 'root_files/lin_-850_500_-10.root', 'root_files/lin_-850_500_-20.root', 'root_files/lin_-850_500_-30.root', 'root_files/lin_-850_500_-40.root'], ['root_files/lin_-850_700_40.root', 'root_files/lin_-850_700_30.root', 'root_files/lin_-850_700_20.root', 'root_files/lin_-850_700_10.root', 'root_files/lin_-850_700_0.root', 'root_files/lin_-850_700_-10.root', 'root_files/lin_-850_700_-20.root', 'root_files/lin_-850_700_-30.root', 'root_files/lin_-850_700_-40.root']]

x_tot, y_tot, pulse_tot = [], [], []
for f in files_tot:
    x_values, y_values, pulse_names = [], [], []
    for dat in f:
        pulse, x_dat, y_dat = collect_pulse_data(dat)
        pulse_names.append(pulse)
        x_values.append(np.array(x_dat))
        y_values.append(np.array(y_dat))
        
    x_tot.append(x_values)
    y_tot.append(y_values)
    pulse_tot.append(pulse_names)

data_tot = array_data(x_tot, y_tot, np.linspace(0.02,5,num=50))
print(data_tot.shape)
#print(np.shape(data_tot))

e_field = collect_efield(files_tot, positions)

#e_field = e_field.reshape(549, 30)
print(e_field.shape)

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

dummy_array1 = np.zeros((len(e_field), 1))
dummy_array2 = np.zeros((len(e_field), 1))

x_train = data_tot.astype("float32")
y_train = e_field.astype("float32")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
y_train = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
print("Min:", y_train.min(), "Max:", y_train.max())
y_train = y_train.reshape(len(e_field), 1)

k = 3
folds = list(KFold(n_splits=k, shuffle=True).split(dummy_array1, dummy_array2))

def build_and_compile_model():
    model = keras.Sequential([
        layers.Dense(50, activation='relu', input_shape=(50,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        #layers.BatchNormalization(),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation= 'linear'),
  ])
    
    model.compile(loss='mse',
            optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['mae'])

    return model

fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()

for j, (train_idx, val_idx) in enumerate(folds):
    
    print('\nFold ',j)
    x_train_cv = x_train[train_idx]
    y_train_cv = y_train[train_idx]
    x_valid_cv = x_train[val_idx]
    y_valid_cv= y_train[val_idx]
    
    model = build_and_compile_model()
    history = model.fit(
            x_train_cv,
            y_train_cv,
            batch_size=56,
            epochs=80,
            shuffle=True,
            verbose=0,
            validation_data = (x_valid_cv, y_valid_cv))

    print(model.evaluate(x_valid_cv, y_valid_cv))
    
    def plot_loss(history):
        fig1, ax1 = plt.subplots()
        ax1.plot(history.history['loss'], label=f'Loss (Fold {j})')
        ax1.plot(history.history['val_loss'], label=f'Validation Loss (Fold {j})')
        #plt.ylim(0,1000)
        #plt.title("Losses for the Machine Learning Model")
        ax1.set_xlabel('Epoch', fontsize=13)
        ax1.set_ylabel('Error', fontsize=13)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        ax1.legend(fontsize=11)
        #plt.grid(True)
        
    plot_loss(history)
    print(history.history['loss'])
    
    test_files1 = [['root_files/lin_300_100_40.root', 'root_files/lin_300_100_30.root', 'root_files/lin_300_100_20.root', 'root_files/lin_300_100_10.root', 'root_files/lin_300_100_0.root', 'root_files/lin_300_100_-10.root', 'root_files/lin_300_100_-20.root', 'root_files/lin_300_100_-30.root', 'root_files/lin_300_100_-40.root']]
    x_test, y_test, pulse_test = [], [], []
    for f in test_files1:
        x_1, y_1, pulse_names = [], [], []
        for dat in f:
            pulse, x, y = collect_pulse_data(dat)
            pulse_names.append(pulse)
            x_1.append(np.array(x))
            y_1.append(np.array(y))
            
        x_test.append(x_1)
        y_test.append(y_1)
        pulse_test.append(pulse_names)


    data_test1 = array_data(x_test, y_test, np.linspace(0.02,5,num=50))
    #print(data_test)
    y_pred1 = model.predict(data_test1)
    y_pred1 = scaler.inverse_transform(y_pred1)
    #print(y_pred)
    
    ax2.plot(positions, y_pred1, label=f'ML Prediction for the Electric Field (Fold {j})')
    #plt.title("Electric Field Prediction using ML Model")
    #plt.ylim(-85000,-45000)
    
    test_files2 = [['root_files/lin_-500_500_40.root', 'root_files/lin_-500_500_30.root', 'root_files/lin_-500_500_20.root', 'root_files/lin_-500_500_10.root', 'root_files/lin_-500_500_0.root', 'root_files/lin_-500_500_-10.root', 'root_files/lin_-500_500_-20.root', 'root_files/lin_-500_500_-30.root', 'root_files/lin_-500_500_-40.root']]#[['root_files/lin_-1000_10_40.root', 'root_files/lin_-1000_10_30.root', 'root_files/lin_-1000_10_20.root', 'root_files/lin_-1000_10_10.root', 'root_files/lin_-1000_10_0.root', 'root_files/lin_-1000_10_-10.root', 'root_files/lin_-1000_10_-20.root', 'root_files/lin_-1000_10_-30.root', 'root_files/lin_-1000_10_-40.root']]
    x_test, y_test, pulse_test = [], [], []
    for f in test_files2:
        x_1, y_1, pulse_names = [], [], []
        for dat in f:
            pulse, x, y = collect_pulse_data(dat)
            pulse_names.append(pulse)
            x_1.append(np.array(x))
            y_1.append(np.array(y))
            
        x_test.append(x_1)
        y_test.append(y_1)
        pulse_test.append(pulse_names)


    data_test2 = array_data(x_test, y_test, np.linspace(0.02,5,num=50))
    #print(data_test)
    y_pred2 = model.predict(data_test2)
    y_pred2 = scaler.inverse_transform(y_pred2)
    #print(y_pred)
    
    ax3.plot(positions, y_pred2, label=f'ML Prediction for the Electric Field (Fold {j})')
    #plt.title("Electric Field Prediction using ML Model")
    ax3.set_ylim(-100000, 50000)
    #plt.ylim(-85000,-45000)
    
ax2.plot(positions, collect_efield(test_files1, positions), label='Allpix Applied Electric Field', color='black')
ax2.set_xlabel('Position (\u03bcm)', fontsize=13)
ax2.set_ylabel('Electric field (V/cm)', fontsize=13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax2.legend(fontsize=11)

ax3.plot(positions, collect_efield(test_files2, positions), label='Allpix Applied Electric Field', color='black')
ax3.set_xlabel('Position (\u03bcm)', fontsize=13)
ax3.set_ylabel('Electric field (V/cm)', fontsize=13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax3.legend(fontsize=11)

plt.show()

In [ ]:
column_names = ['Current_pulse', 'E_field']

x_train = data_tot.astype("float32")
y_train = e_field.astype("float32")

def build_and_compile_model():
  model = keras.Sequential([
    layers.Dense(1000, activation='relu'),
    layers.Dense(1000, activation='relu'),
    layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

model = build_and_compile_model()

history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    verbose=0, epochs=175)



In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  #plt.ylim(0,1000)
  plt.title("Losses for the Machine Learning Model")
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
  
plot_loss(history)
print(history.history['loss'])

In [ ]:
test_files = [["root_files/lin4_n40.root", "root_files/lin4_n30.root", "root_files/lin4_n20.root", "root_files/lin4_n10.root", "root_files/lin4_0.root", "root_files/lin4_p10.root", "root_files/lin4_p20.root", "root_files/lin4_p30.root", "root_files/lin4_p40.root"]]
x_test, y_test, pulse_test = [], [], []
for f in test_files:
    x_1, y_1, pulse_names = [], [], []
    for dat in f:
        pulse, x, y = collect_pulse_data(dat)
        pulse_names.append(pulse)
        x_1.append(np.array(x))
        y_1.append(np.array(y))
        
    x_test.append(x_1)
    y_test.append(y_1)
    pulse_test.append(pulse_names)

data_test = array_data(x_test, y_test, np.linspace(0.1,5,num=100))
#print(data_test)
y = model.predict(data_test)
print(y)

In [ ]:
print(y[0])
plt.plot(positions, collect_efield(test_files, positions)[0], label='Allpix Applied Electric Field')
plt.plot(positions, y[0], color='k', label='ML Prediction for the Electric Field')
plt.title("Electric Field Prediction using ML Model")
plt.xlabel('Position (\u03bcm)')
plt.ylabel('E_field (V/cm)')
#plt.ylim(-85000,-45000)
plt.legend()
plt.show()